<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Notation" data-toc-modified-id="Notation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Notation</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Hyperparameters" data-toc-modified-id="Hyperparameters-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hyperparameters</a></span></li></ul></div>

# Neuronal Network (NN)
Basically, a NN is an entity that is able to learn (something) a number of dependencies between the inputs and the outputs of a system.

A NN is a collection of neurons, organized in layers. In the direction of the propagation of the information, (from the input to the output of the network) the first one is called the input layer and the last one the output layer. The rest of layers are said hidden.

The number of neurons in the input and the output layers, that can be any, are defined by the problem we want to address. The number of hidden layers and the number of neurons/layer depends on the complexity of the dependiencias to learn.

# Notation
* $l=0,\cdots,L-1$ the layer index.
* $a_i^l; i=0,\cdots$ the activation of the $i$-th neuron of the $l$-th layer. By definition, $a^{L-1}=y$, the output activation of the NN.
* $w_{ij}^l$ the weight that goes from the $j$-th neuron of the $l$ layer to the $i$-th neuron of the layer $l+1$. $b_{i}^l$ the bias that goes from the $i$-th neuron of the $l$ layer.
* $e_i^l$ the scalar error of the $i$-th neuron of the $l$-th layer. By definition, the prediction error $e^{L-1}=t-y$, where $t$ is the target (ideal) output of the NN. The error in (the activation of) the internal neurons $e_i^l; l=1, \cdots, L-2$ is estimated by using the Back-Propagation Algorithm (BPA).
* $e_i^l=t_i-a_i^{L-1}$, the error of the $i$-th neuron of the $l$-th layer, where $t_i$ is the target value for the $i$-th component of the desired output $y$. Notice that $e^{L-1}$ is the error of the network.
* By convenience, the $e^{L-1}$ is not directly minimized, but the cost function $c=\sum_i e_i^2$ .

# Implementation
Based on http://neuralnetworksanddeeplearning.com

In [105]:
import numpy as np
#import ipdb

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_derivative(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

class Network:

    def __init__(self, sizes=[2, 3, 1], learning_rate=1, b_init_vals=-1, w_init_vals=-1):
        self.num_layers = len(sizes)
        self.sizes = sizes
        
        if b_init_vals == -1:
            self.B = [np.random.rand(y, 1) for y in sizes[1:]]
        else:
            assert b_init_vals >= 0.0
            self.B = [np.full((y, 1), b_init_vals) for y in sizes[1:]]
        
        if w_init_vals == -1:
            self.W = [np.random.rand(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        else:
            assert w_init_vals >= 0.0
            self.W = [np.full((y, x), w_init_vals) for x, y in zip(sizes[:-1], sizes[1:])]
        
        #for x, y in zip(sizes[:-1], sizes[1:]):
        #    print (x, y)
        self.LR = learning_rate

    def feed_forward(self, a):
        '''Propagates an activation ``a`` from the input to the output of the network.'''
        for b, w in zip(self.B, self.W):
            print(w.shape, a.shape)
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def learn(self, x, t):
        '''NN learn when the weights and biases are modified to minimize the cost function.
        To teach a NN, tuples at least a tuple ```(x, y)``` must be presented
        to the NN, where ```x``` is a input training example (also called a feature vector)
        and ``t``` is the associated (ideal) output (target) that the NN should learn.
        
        To modify the weights and the biases, Gradient Descend Optimization (GDO) is used.
        '''
        
        # Find the gradient for each weight and bias of the NN
        nabla_b, nabla_w = self.get_gradients(x, t)
        
        # w_ij^l -= \alpha/len(x)\nabla_w_ij^l
        self.W = [w - (self.LR/len(x))*nw for w, nw in zip(self.W, nabla_w)]
        
        # b_i^l -= \alpha/len(x)\nabla_b_i^l
        self.B = [b - (self.LR/len(x))*nb for b, nb in zip(self.B, nabla_b)]

    def cost_derivative(self, y, t):
        """Derivative of the cost function: 1/2*(ideal_out - out)^2.
        Notice that the scalar has been ignored (supposed to be 1) it will be
        considerated to be a part of the learning rate used by GDO."""
        return (y - t)

    def get_gradients(self, _in, ideal_out):
        """ Given an input ``_in`` and an ideal output ``ideal_out``,
        modify the weights and biases to minimize the cost of the error.
        
        Backpropagate the errors from the output to the first hidden layer,
        and computes the 
        """
        #ipdb.set_trace() # <-------------------------- breakpoint
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.B`` and ``self.W``."""
        
        # Returns the gradient of the cost function C(x) respect to the
        # biases (nabla_b) and weights (nabla_w).
        nabla_b = [np.zeros(b.shape) for b in self.B]
        nabla_w = [np.zeros(w.shape) for w in self.W]
        #print(len(nabla_b), len(nabla_w))

        # Forward pass. We compute two lists: activations and zs,
        # with the activations and the z's of the neurons of the network.
        activation = _in
        activations = [_in] # list to store all the activations, layer by layer
        #print(_in)
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.B, self.W):
            #for i in activations:
            #    print(i.shape)
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)

        # Backward pass. This is the backpropagation algorithm.
        
        # Derivative of the error of the cost function at the output (L-1) layer.
        delta = self.cost_derivative(activations[-1], ideal_out) * sigmoid_derivative(zs[-1])
        
        # The gradient of the cost function respect to the biases at the output layer
        # is the calculus performed in the last sentence.
        nabla_b[-1] = delta
        
        # The gradient of the cost function respect to the weights at the output
        # is the previous derivative multiplied by the activations of the L-2 layer.
        #print(delta.shape, activations[-2].transpose().shape)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        # Now, retropropagate the error of the cost function to the rest of the 
        # layers (starting at L-2) up to the first one (layer 0), computing the gradient.
        for l in range(2, self.num_layers):
            z = zs[-l] # Negative indexes go backwards in the list
            sp = sigmoid_derivative(z)
            delta = np.dot(self.W[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())

        return (nabla_b, nabla_w)

# Hyperparameters

* Input layer: $s_0$ and $s_2$, samples of $s_0, s_1, s_2, \cdots$. 
* Output layer: $\hat{s}_1$, a prediction.
* Initial prediction:
$$
\hat{s}_1 = \frac{s_0 + s_2}{2}.
$$
* $L$ layers and number of neurons by layer.

In [107]:
net = Network(sizes=[2, 16, 16, 1], b_init_vals=-1, w_init_vals=-1)
for i in range(2000):
    net.learn(np.array([[10/255],[50/255]]), np.array([[130/255]]))
print(net.feed_forward(np.array([[10/255],[50/255]])) * 255)

(16, 2) (2, 1)
(16, 16) (16, 1)
(1, 16) (16, 1)
[[130.]]


In [108]:
net = Network(sizes=[2, 16, 16, 1], b_init_vals=0, w_init_vals=0)
for i in range(2000):
    net.learn(np.array([[10/255],[50/255]]), np.array([[130/255]]))
print(net.feed_forward(np.array([[10/255],[50/255]])) * 255)

(16, 2) (2, 1)
(16, 16) (16, 1)
(1, 16) (16, 1)
[[130.]]


In [73]:
x = np.random.randint(low=0, high=100, size=(2,3))
x

array([[41, 55, 37],
       [74, 16, 94]])

In [63]:
x[1,0]

29